MOTIVATION

Lossless compression reduces the stored size of data by transforming to an alternate representation. In lossless compression schemes the input can be reconstructed perfectly, unlike lossy methods which may lose some data which is considered 'less important'.

Typical compression utilities include GZip and XZ, which are both sliding window dictionary based methods. These algorithms exploit repetition in text, but do not consider the content of the text itself. Common phrases and patterns which exist in natural language however are not explicitly leveraged. In this sense some information lays outside of the data itself, embedded in the language.

Bidirectional Encoder Representations from Transformers (BERT) is a language model from Google, which uses a bidirectional transformer pre-trained on a large corpus using significant computing power. BERT has been shown to be exceed prior models on several benchmarks including question answering and next sentence prediction. We explore the potential of BERT to enhance existing lossless compression techniques.

---
APPROACH

Given an input sentence, we aim to substitute a subset of the sentence with placeholders to improve the efficacy of subsequent compression. In order to achieve lossless compression, we ensure that the operation is invertible before performing the replacement.

The BERT model is used for both compression and decompression tasks. The input text is is first tokenized in two phases to preserve mappings from token to input text. The basic tokenization is easily mapped back to the original text, while the WordPiece tokenization is required for pre-trained BERT. Compression involves determining a combination of tokens which can be masked and predicted with BERT to recover the original sentence. Any such sentence is considered to be a valid substitution. We configure BERT to use one previous sentence along with the current sentence for context. Since decompression is performed on sentence in order, the previous sentence is always available in uncompressed form.

Checking the invertibility of substitution is computationally intensive, thus it is important to minimize use of this operation. At the same time, it is desirable have more of the sentence masked to achieve higher compression. Note that some maskings will produce higher compression than others even if the length or count of the words is identical. This is a property of the subsequent coding methods, and will differ based on the methods used. 

Although it is possible to check all combinations of mask configurations for a given sentence, this will result in excessive runtime and is therefore undesirable. We therefore propose two heuristics to approximate a best solution.
The first heuristic tries to remove words by length. We found preferring to remove short words over long words has better performance. It tends to remove words such as “the” and “to”.
The second heuristic tries to remove words by impact. Impact is a measure we invented to describe the relationship a word has on other words. The impact, of a word w is how how many more or less words can be guessed when word w is removed. We prefer to remove words with high impact, or where the set of words that can be guessed afterwards is large. Unfortunately getting the impact of each word is costly, and the impact of all the words must be recomputed when a word is removed.

Once a combination of mask positions is selected, word substitution is performed on the original text. Each mask is replaced with a placeholder symbol which represents a word to be masked when performing decompression. The substitute text is then compressed using well known coding methods to achieve the final compressed output. We use the Huffman coding algorithm as well as the GZip and XZ utilities to verify improvement of final compression ratios, however this technique should extend to other frequency based algorithms.

---
DATA

We download randomized pages from Wikipedia and concatenate them together to form 5 datasets of 10,000 words each. We use the freely available Wikipedia API to perform the queries. We select 10,000 words in order to limit the runtime of compression. We compressed both large and small data using Huffman, Gzip, and XZ without BAWS to confirm that compression ratios are relatively stable down to our selected dataset size of 10,000 words.

```
pip install wikipedia
```

---
CODE

- Huffman coding, we used someone else's implementation, and change slightly
    - https://github.com/bhrigu123/huffman-coding.
- XZ and GZip easily installed on Ubuntu 
- use prebuilt BERT model
   - https://github.com/huggingface/pytorch-pretrained-BERT
   - wrote our own interface to use
       - `MaskEvaluator` and `ReversibleBertTokenizer` classes
- deciding what words to mask is “difficult”
   - wrote our own code to choose what words to mask
       - get_by_length and get_by_impact functions



```
sudo apt-get install gzip
sudo apt-get install xz
pip install pytorch-pretrained-bert
```

---
EXPERIMENTAL SETUP

We run standard compression without substitution on the 5 datasets, averaging the results to obtain a baseline compressed size. We then repeat the process for the substituted text (BAWS) and compare the final compressed size to the baseline compressed size.

---
RESULTS

To the best of our knowledge, prior work on language model based word compression is limited. One comparable paper https://nlp.stanford.edu/courses/cs224n/2006/fp/aeldaher-jconnor-1-report.pdf achieves compression of approximately 30%, however we we unable to obtain the dataset for a direct comparison. Small datasets were also used in this paper due to long runtimes.

We found that using BAWS results in approximately 25% of words in a sample text to be replaced however this does not measure the actual efficacy in a compression context. To ensure that entropy is reduced with respect to common algorithms, we measure the final compressed size both with and without BAWS as a percent difference between compressed, and compressed with BAWS.

$\frac{\text{basic_compression_size - baws_compression_size}}{\text{basic_compression_size}}$

Huffman coding benefits most from BAWS. We found BAWS to reduce the final compressed size by an additional 32.54%. The GZip and XZ utilities used in conjunction with BAWS results in 24.03% and 24.61% difference respectively.


	
---


ANALYSIS OF THE RESULTS

The difference between the character and sequence based compression methods seems to indicate that the character entropy is reduced more than the word entropy. Further investigation is required to identify the source of these observed differences.

The runtime to compress 10,000 words (8th Gen Intel I7 laptop) is approximately 30 minutes. This runtime prevented evaluation of larger files.



---
FUTURE WORK

One significant issue with BAWS is the long runtimes associated with selecting words to mask. The checking of invertibility is computationally expensive since the model must be evaluated. By identifying valid combinations with less attempts, a significant speedup should be observed. It is likely that this problem can be modeled probabilistically, replacing the proposed “get by length” and “get by impact” heuristics.



In [0]:
%%capture
!pip install pytorch-pretrained-bert
!pip install numpy
!pip install wikipedia

In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM

import itertools
from copy import deepcopy
import numpy as np
import wikipedia
import time

from collections import defaultdict

import heapq
import os
from functools import total_ordering

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


This class **MaskEvaluator**  manages the BERT model. The main functionality of the class is to 
1.   Tokenize sentences
2.   Determine if a group of indexes can be removed from a sentence.

`me.evaluate` returns an array where the first index is `True` if the group of indexes can be removed from the sentence. The second index is an array that returns what indexes should be removed to return `True` (if already true an empty array is returned).

EXAMPLE
```
text = 'i enjoy doing NLP. hopefully my group will do well on the project.'

# the init will tokenize the sentence
me = MaskEvaluator(text)

print('')
sentence_num = 1
indexes_to_mask = [0, 2, 5]
np_sentence = np.array(me.sentences[sentence_num])
np_indexes = np.array(indexes_to_mask)
sentence = ' '.join(me.sentences[sentence_num])
words = np.array(me.sentences[1])[np.array(indexes_to_mask)]

# calling me.evaluate will determine if a group of words can be removed
if me.evaluate(sentence_num, indexes_to_mask)[0]:
    print('From [{sentence}], the words {words} can be masked'
          .format(sentence=sentence, words=words))
else:
    print('From [{sentence}], the words {words} cannot be masked'
          .format(sentence=sentence, words=words))
```
--> * `From [hopefully my group will do well on the project .], the words ['hopefully' 'group' 'well'] cannot be masked` *

**Choosing which group of words to remove is non-trivial.** For example, in the example the indexes 
`[0, 2, 5]` were chosen arbitrarily. Given infinite time, we would like to try all cases and pick the
largest group of indexes. That is, if we could pick `[0, 5]` or `[0, 1, 3]`, we would pick the
second one because it's length (3) is bigger.

This problem will be explored later on in the notebook.

In [0]:
class ReversibleBertTokenizer(BertTokenizer):
    '''
    Extend the BertTokenizer to provide reversibilty
    '''
    def reversible_tokenize(self, text):
        basic_tokens = self.basic_tokenizer.tokenize(text)
        split_tokens = []
        basic_index_map = {}
        split_index_map = defaultdict(list)
        for index, token in enumerate(basic_tokens):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                # Map the basic token index to the split token index
                basic_index_map[len(split_tokens)] = index
                # Map split tokens to a basic token
                split_index_map[index].append(len(split_tokens))
                # Append the subtoken
                split_tokens.append(sub_token)

        return split_tokens, basic_tokens, split_index_map, basic_index_map


'''
Evaluate a sentence masking configuration
'''
class MaskEvaluator:
    def __init__(self, text):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = ReversibleBertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
        self.model = BertForMaskedLM.from_pretrained('bert-base-cased')

        # Tokenize
        self.split_tokens, self.basic_tokens, self.split_index_map, self.basic_index_map = self.tokenizer.reversible_tokenize(text)

        # Get sentence start indices
        self.sentence_indices = [0]
        for index, basic_token in enumerate(self.basic_tokens):
            # TODO: Make these splits better
            if basic_token == '.':
                self.sentence_indices.append(index+1) # Point at the start of the next sentence

        # TODO: This can be removed eventually. Currently will break application
        self.sentences = [] # Only used by appliation code. Not required
        for i in range(len(self.sentence_indices)-1):
            self.sentences.append(self.get_sentence(i))

    def get_count(self):
        return len(self.sentences)

    def get_sentence(self, index):
        return deepcopy(self.basic_tokens[self.sentence_indices[index]:self.sentence_indices[index+1]])

    def get_sentence_offset(self, index):
        return self.sentence_indices[index]

    def get_split_sentence(self, index):
        return self.get_masked_split_sentence(index)

    def get_masked_split_sentence(self, index, mask=None):
        if mask is None:
            mask = []
        basic_offset = self.get_sentence_offset(index)
        split_sentence = []
        for basic_index, basic_token in enumerate(self.get_sentence(index)):
            # Mask out all split tokens if basic token is masked
            if basic_index in mask:
                split_sentence.extend(['[MASK]' for _ in 
                    range(len(self.split_index_map[basic_offset+basic_index]))])
            else:
                split_sentence += deepcopy(
                        [self.split_tokens[i] for i in 
                            self.split_index_map[basic_offset+basic_index]])
        return split_sentence

    '''
    Evaluate with previous sentence context.
    index -> target sentence index.
    mask -> a list of indicies at which a mask should be placed. This requests
        word replacement at this index. Mask indicies that do not exist are ignored.
    '''
    def evaluate(self, index, mask):
        previous = [] if index == 0 else self.get_split_sentence(index-1)
        current = self.get_split_sentence(index)
        masked = self.get_masked_split_sentence(index, mask)

        # Convert indicies
        indexed = self.tokenizer.convert_tokens_to_ids(previous + masked)

        # Define seperator mask for sentence A and B
        segment_a = [0 for _ in range(len(previous))]
        segment_b = [1 for _ in range(len(masked))]

        # Convert inputs to tensors
        tokens = torch.tensor([indexed])
        segments = torch.tensor([segment_a + segment_b])

        # Set model to evaluation mode
        self.model.eval()

        # Make predictions
        predicted = self.model(tokens, segments)
        predicted_indices = [torch.argmax(
                predicted[0, i+len(previous)]).item() for i in range(len(masked))]
        predicted_tokens = self.tokenizer.convert_ids_to_tokens(predicted_indices)

        # Validate predictions
        invalid = []
        for i in mask:
            if predicted_tokens[i] != current[i]:
                invalid.append(i)
        
        return bool(len(invalid) == 0), invalid

The class **HuffmanCoding** does Huffman coding. The code has been heavily borrowed from 
**https://github.com/bhrigu123/huffman-coding**. The code has been edited to fit our task. 

Huffman coding works by assigning variable length codes to characters. Characters that frequently
appear will be assigned short length codes.

For instance, suppose we had the word `hello`. We would assign the shortest possible length code to `l` because it is the most frequent character.

EXAMPLE
```
hc = HuffmanCoding('')
text = 'i enjoy doing NLP. hopefully my group will do well on the project.'
hc.commpress_text(text, 'compressed_text.bin')
```

In [0]:
"""
Code for Huffman Coding, compression and decompression. 
Explanation at http://bhrigu.me/blog/2017/01/17/huffman-coding-python-implementation/
"""


@total_ordering
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    # defining comparators less_than and equals
    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if (other == None):
            return False
        if (not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq


class HuffmanCoding:
    def __init__(self, path):
        self.path = path
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}

    # functions for compression:

    def make_frequency_dict(self, text):
        frequency = {}
        for character in text:
            if not character in frequency:
                frequency[character] = 0
            frequency[character] += 1
        return frequency

    def make_heap(self, frequency):
        for key in frequency:
            node = HeapNode(key, frequency[key])
            heapq.heappush(self.heap, node)

    def merge_nodes(self):
        while (len(self.heap) > 1):
            node1 = heapq.heappop(self.heap)
            node2 = heapq.heappop(self.heap)

            merged = HeapNode(None, node1.freq + node2.freq)
            merged.left = node1
            merged.right = node2

            heapq.heappush(self.heap, merged)

    def make_codes_helper(self, root, current_code):
        if (root == None):
            return

        if (root.char != None):
            self.codes[root.char] = current_code
            self.reverse_mapping[current_code] = root.char
            return

        self.make_codes_helper(root.left, current_code + "0")
        self.make_codes_helper(root.right, current_code + "1")

    def make_codes(self):
        root = heapq.heappop(self.heap)
        current_code = ""
        self.make_codes_helper(root, current_code)

    def get_encoded_text(self, text):
        encoded_text = ""
        for character in text:
            encoded_text += self.codes[character]
        return encoded_text

    def pad_encoded_text(self, encoded_text):
        extra_padding = 8 - len(encoded_text) % 8
        for i in range(extra_padding):
            encoded_text += "0"

        padded_info = "{0:08b}".format(extra_padding)
        encoded_text = padded_info + encoded_text
        return encoded_text

    def get_byte_array(self, padded_encoded_text):
        if (len(padded_encoded_text) % 8 != 0):
            print("Encoded text not padded properly")
            exit(0)

        b = bytearray()
        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i + 8]
            b.append(int(byte, 2))
        return b

    def compress_text(self, text, output_path):
        text = ''.join(text)
        frequency = self.make_frequency_dict(text)
        self.make_heap(frequency)
        self.merge_nodes()
        self.make_codes()
        encoded_text = self.get_encoded_text(text)
        padded_encoded_text = self.pad_encoded_text(encoded_text)
        b = self.get_byte_array(padded_encoded_text)
        with open(output_path, 'wb') as output:
            output.write(bytes(b))
        # return padded_encoded_text

    def compress_file(self):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + ".bin"

        with open(self.path, 'r+') as file, open(output_path, 'wb') as output:
            text = file.read()
            text = text.rstrip()

            frequency = self.make_frequency_dict(text)
            self.make_heap(frequency)
            self.merge_nodes()
            self.make_codes()

            encoded_text = self.get_encoded_text(text)
            padded_encoded_text = self.pad_encoded_text(encoded_text)

            b = self.get_byte_array(padded_encoded_text)
            output.write(bytes(b))

        print("Compressed")
        return output_path

    """ functions for decompression: """

    def remove_padding(self, padded_encoded_text):
        padded_info = padded_encoded_text[:8]
        extra_padding = int(padded_info, 2)

        padded_encoded_text = padded_encoded_text[8:]
        encoded_text = padded_encoded_text[:-1 * extra_padding]

        return encoded_text

    def decode_text(self, encoded_text):
        current_code = ""
        decoded_text = ""

        for bit in encoded_text:
            current_code += bit
            if (current_code in self.reverse_mapping):
                character = self.reverse_mapping[current_code]
                decoded_text += character
                current_code = ""

        return decoded_text

    def decompress(self, input_path):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + "_decompressed" + ".txt"

        with open(input_path, 'rb') as file, open(output_path, 'w') as output:
            bit_string = ""

            byte = file.read(1)
            while (len(byte) > 0):
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = file.read(1)

            encoded_text = self.remove_padding(bit_string)

            decompressed_text = self.decode_text(encoded_text)

            output.write(decompressed_text)

        print("Decompressed")
        return output_path


Recall **choosing which group of words to remove is non-trivial.** The problem is sentences of length N has

\begin{equation*}
\left( \sum_{k=1}^N \binom{N}{k} \right)
\end{equation*}

possible removable groups.

Here we will explore different algorithms that will determine which group of words to choose. All of them will attempt to maximize the length of the group being removed. There are **definitely** better methods to determine the best group to remove. All of the algorithms listed follow a *greedy* approach. 
We only add indexes to the group so that using `me.evaluate` on the group returns `True` (recall 
this means we can still retrieve the original sentence). 

This approach is flawed. Consider 
```
group1 = [0]
group2 = [1]
group3 = [0, 1]
```
It is possible `me.evaluate` returns `False` for `group1` and `group2`, but returns `True` for `group3`. Our algorithm would never get to `group3` in this case. However, we found our current methods are "good enough" by helping decompress by an additional ~25%.

---

Algorithm 1: **get_by_length** This algorithm sorts the words by length, and tries to remove indicies from the list in that order. Experiments show removing shortest or longest words first does not significantly change the number of words removed.

---

Algorithm 2: **get_by_impact** This algorithm sorts the words by impact, and tries to remove indicies from the list in that order. Impact is a definition we invented. The impact if an index given a sentence and indexes, is how much an index affects the ability of the BERT algorithm to retrieve the original sentence.

For example, given a sentence = [w1, w2, w3, s4], and indexes = [2, 3, 4], we would like to measure the impact of the indexes. Recall the second value returned from `me.evaluate` is which indexes should be removed from the original indexes, for `me.evaluate` to return `True`
*   impact of 2 is has value `len(me.evaluate(sentence, [3, 4])[1])`
*   impact of 3 is has value `len(me.evaluate(sentence, [2, 4])[1])`
*   impact of 4 is has value `len(me.evaluate(sentence, [2, 3])[1])`

We can sort be largest impact or smallest impact. Our experiments show **sorting by largest impact outperforms the other algorithms on amount compressed.** However, **sorting by shortest word is much faster and performs similarily to largest impact**.


In [0]:
def get_single_index(me, s, sentence):
    # see which words can be removed by itself
    sentence_order = []
    for c, word in enumerate(sentence[-1:]):
        value = me.evaluate(s, [c])
        if value[0]:
            sentence_order.append(c)
    return sentence_order


def get_by_length(me, sort_by='largest'):
    if sort_by not in ['largest', 'smallest']:
        raise ValueError("[sort_by] must be largest or smallest")
    removable = []
    sent_length = len(me.sentences)
    for s, sentence in enumerate(me.sentences):
        print(s/sent_length)
        word_counter_ = []
        
        # sort by length of each word
        for i, word in enumerate(sentence):
            word_counter_.append([i, len(word)])
        word_counter_.sort(key=lambda x: x[1])
        if sort_by == 'largest':
            word_counter_.reverse()
        word_counter = []
        for arr in word_counter_:
            word_counter.append(arr[0])

        # see which words can be removed (sorted by length)
        all_index_good = {}
        for i, ind in enumerate(word_counter):
            if me.evaluate(s, [ind])[0]:
                all_index_good = {ind}
                word_counter.pop(i)
        if len(all_index_good) == 0:
            removable.append(all_index_good)
            continue

        # see which words can be removed
        # just go in order of appearance in sentence
        prev_length = -1
        curr_length = len(all_index_good)
        while prev_length != curr_length:
            prev_length = len(all_index_good)
            for i, ind in enumerate(word_counter):
                if me.evaluate(s, [ind])[0]:
                    all_index_good.add(ind)
                    word_counter.pop(i)
            curr_length = len(all_index_good)
        removable.append(all_index_good)
    return removable


def get_by_impact(me, sort_by='largest', indexes=None):
    if sort_by not in ['largest', 'smallest']:
        raise ValueError("[sort_by] must be largest or smallest")
    removable = []
    for s, sentence in enumerate(me.sentences):
        # find single indexes that can be removed
        single_index_replaceable = set(get_single_index(me, s, sentence))
        for index in single_index_replaceable:
            if len(sentence[index]) == 1:
                single_index_replaceable = single_index_replaceable - {len(sentence)-1}
        value = me.evaluate(s, single_index_replaceable)
        if len(single_index_replaceable) == 0:
            removable.append({})
        if value[0]:
            removable.append(single_index_replaceable)
            continue
        invalid_counter_ = []
        
        # sort words by impact
        for single_index in single_index_replaceable:
            test_indexes = single_index_replaceable - {single_index}
            num_invalid = len(me.evaluate(s, test_indexes)[1])
            invalid_counter_.append([single_index, num_invalid])
        invalid_counter_.sort(key=lambda x: x[1])
        if sort_by == 'largest':
            invalid_counter_.reverse()
        invalid_counter = []
        for arr in invalid_counter_:
            invalid_counter.append(arr[0])
        all_index_good = {invalid_counter.pop(0)}
        iter_max = int(np.log(len(invalid_counter))+1)
        iter = 0
        
        # from the indexes that can be individually removed, try seeing which groups can be removed
        all_index_good = __get_by_impact_helper(me, s, all_index_good, invalid_counter, iter_max, iter)

        # from the indexes that cannot be individually removed, try adding to the above group
        all_index_bad = list(set(np.arange(len(sentence))) - all_index_good - set(invalid_counter))
        all_index_good_ = __get_by_impact_helper(me, s, all_index_good, all_index_bad, iter_max, iter)
        if len(all_index_good_) > len(all_index_good):
            all_index_good.update(all_index_good_)
            all_index_good = __get_by_impact_helper(me, s, all_index_good, invalid_counter, iter_max, iter)
        removable.append(all_index_good)
    return removable


def __get_by_impact_helper(me, s, all_index_good, invalid_counter, iter_max, iter):
    if iter >= iter_max:
        return all_index_good
    iter += 1
    remove_array = []
    for i, index in enumerate(invalid_counter):
        temp_all_index_good = all_index_good.union({index})
        if me.evaluate(s, temp_all_index_good)[0]:
            all_index_good = temp_all_index_good
            remove_array.append(i)
    for i in sorted(remove_array, reverse=True):
        del invalid_counter[i]
    if len(remove_array) > 0:
        all_index_good.update(__get_by_impact_helper(me, s, all_index_good, invalid_counter, iter_max, iter))
    return all_index_good


def measure_by_words_removed(token_paragraph, count):
    alpha = 0
    for myset in token_paragraph:
        alpha += len(myset)
    print(alpha/count)
    return alpha/count

def mask_sentences(sentences, removables):
    tag_holder = '^'
    sentences_ = []
    for i, sentence in enumerate(sentences):
        removable = removables[i]
        for j, word in enumerate(sentence):
            if j in removable:
                if word[:2] == '##':
                    sentences_[-1] += tag_holder
                else:
                    sentences_.append(tag_holder)
            else:
                if word[:2] == '##':
                    sentences_[-1] += word[2:]
                else:
                    sentences_.append(word)
    return sentences_


The dataset to download the data is done here. Wikipedia pages are of variable length. We choose to  stop downloading  wikipedia pages when the total amount of words is greater than 10,000.

In [0]:
def get_wiki_articles(length=20):
    word_arr_all = []
    error_counter = 0
    while len(word_arr_all) < length:
        try:
            content = wikipedia.page(wikipedia.random()).content.replace('\n', ' ')
            content_split = content.split(' ')
            if content_split[-1][-1] != '.':
                content_split[-1] = content_split[-1] + '.'
            word_arr_all += content.split(' ')
        except:
            continue
    word_arr = word_arr_all[:length]
    if word_arr[-1] != '.':
        word_arr.append('.')
    return word_arr
        
def save_text_files(word_arr, folder='.'):
    # save the raw text file
    with open(folder+'/wiki.txt', 'w') as output:
        output.write(' '.join(word_arr))

    # do HuffmanCoding on the raw text, and save it
    h = HuffmanCoding('')
    h.compress_text(word_arr, folder+'/wiki_org.bin')

def save_compress_files(me, folder='.'):
    length_smallest = get_by_length(me, sort_by='smallest')
    length_smallest_sents = mask_sentences(deepcopy(me.sentences), length_smallest)
    with open(folder+'/wiki_length_small.txt', 'w') as output:
        output.write(' '.join(length_smallest_sents))    
    h = HuffmanCoding('')
    h.compress_text(length_smallest_sents, folder+'/wiki_length_small.bin')


In [7]:
try:
    os.mkdir('results')
except FileExistsError:
    pass
# length=1000
length=10
for i in range(5):
    word_arr = get_wiki_articles(length=length)
    folder = 'results/{i}'.format(i=i)
    try:
        os.mkdir(folder)
    except FileExistsError:
        pass
        # Tokenize the sentences
    word_str = ' '.join(word_arr)  #.replace('\n', ' ')
    me = MaskEvaluator(word_str)

    save_text_files(word_arr, folder=folder)
    save_compress_files(me, folder=folder)


100%|██████████| 404400730/404400730 [00:10<00:00, 37246534.90B/s]


0.0
0.0
0.0
0.5
0.0
0.5


/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


0.0


In [8]:
!zip -r results.zip results/

  adding: results/ (stored 0%)
  adding: results/3/ (stored 0%)
  adding: results/3/wiki_length_small.bin (stored 0%)
  adding: results/3/wiki_length_small.txt (deflated 12%)
  adding: results/3/wiki_org.bin (stored 0%)
  adding: results/3/wiki.txt (stored 0%)
  adding: results/0/ (stored 0%)
  adding: results/0/wiki_length_small.bin (stored 0%)
  adding: results/0/wiki_length_small.txt (deflated 23%)
  adding: results/0/wiki_org.bin (stored 0%)
  adding: results/0/wiki.txt (deflated 8%)
  adding: results/2/ (stored 0%)
  adding: results/2/wiki_length_small.bin (stored 0%)
  adding: results/2/wiki_length_small.txt (deflated 19%)
  adding: results/2/wiki_org.bin (stored 0%)
  adding: results/2/wiki.txt (deflated 8%)
  adding: results/1/ (stored 0%)
  adding: results/1/wiki_length_small.bin (stored 0%)
  adding: results/1/wiki_length_small.txt (deflated 10%)
  adding: results/1/wiki_org.bin (stored 0%)
  adding: results/1/wiki.txt (deflated 7%)
  adding: results/4/ (stored 0%)
  adding: 